In [163]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [164]:
train = pd.read_csv('train.csv') # /kaggle/input/nlp-getting-started/train.csv
test = pd.read_csv('test.csv') # /kaggle/input/nlp-getting-started/test.csv
#sample_submission = pd.read_csv('/kaggle/input/nlp-getting-started/sample_submission.csv')

In [165]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [166]:
test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [167]:
train['text'].sample(30)

1638    + DID YOU SAY TO HIM!!?!?!?!' and phil actuall...
69      Accident center lane blocked in #SantaClara on...
6579    Molecularly targeted cancer therapy for his #L...
3857                          watching it go up in flames
6       #flood #disaster Heavy rain causes flash flood...
7359    Windstorm lastingness perquisite - acquiesce i...
7495       Greer man dies in wreck http://t.co/n2qZbMZuly
5958    / it's fine baby I was screaming at the TV x h...
3806    Former Township fire truck being used in Phili...
3663                               @Truly_Stings Yo Dm me
7580    #FX #forex #trading Cramer: Iger's 3 words tha...
4652    As a result of heavy rains in #Bangladeshaffec...
6657    Three Israeli soldiers wounded in West Bank te...
6667    Purple Heart Vet Finds Jihad Threat on His Car...
2540    Imagini noi si 2 clipuri The Hobbit: The Desol...
4084    All Hail Lady Cheryl!! 24 Reasons You Should B...
6253    Photo: mothernaturenetwork: What is thundersno...
4876    @blair

In [168]:
train['keyword'].isnull().values.sum()

61

In [169]:
train['location'].isnull().values.sum()

2533

In [170]:
train['text'].isnull().values.any()

False

In [171]:
# Drop rows that have missing keyword
##
train = train.dropna(axis = 0,subset=['keyword'])

In [172]:
train.head()

,id,keyword,location,text,target
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0


**Text processing require:**  
- remove punctuation  
- remove numbers  
- to lowercase  
- remove stopwords  
- lemmatize  
- remove non-english words  

**NOTE** : The order of the steps matter. The text processing function should execute steps in this outlined order.

In [291]:
# A function to process text as outlined above

import re
import string
import nltk
from nltk.corpus import stopwords

#words = nltk.download('words')
words = set(nltk.corpus.words.words())

regex = re.compile('[^a-zA-Z]') # remove emojis and any non-letters

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def text_process(text):
    cln = [c for c in text if c not in string.punctuation]
    cln = ''.join(cln)
    cln = [c for c in cln if not c.isdigit()]
    cln = ''.join(cln)
    cln = cln.lower()
    cln = [word for word in cln.split() if word not in stopwords.words('english')]
    cln = ' '.join(cln)
    cln = [lemmatizer.lemmatize(word) for word in nltk.word_tokenize(cln)]
    cln = ' '.join(cln)
    cln = regex.sub(' ', cln)
    cln = [word for word in nltk.wordpunct_tokenize(cln) if word in words or not word.isalpha()]
    return cln #[word for word in cln.split()]

In [292]:
from sklearn.feature_extraction.text import CountVectorizer

bow_transformer = CountVectorizer(analyzer=text_process)
bow_transformer.fit(train['text'])

CountVectorizer(analyzer=<function text_process at 0x7f4227cf23b0>,
                binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [293]:
len(bow_transformer.vocabulary_)

6693

In [294]:
print(bow_transformer.transform([train['text'][1230]]))

  (0, 327)	1
  (0, 1049)	1
  (0, 2100)	1
  (0, 2249)	1
  (0, 2699)	1
  (0, 3019)	1


In [295]:
bow_transformer.get_feature_names()[1048]

'civilization'

In [296]:
bow_text = bow_transformer.transform(train['text'])

In [297]:
bow_text.shape

(7552, 6693)

In [298]:
#tfidf

from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer()
tfidf_transformer.fit(bow_text)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [299]:
print(tfidf_transformer.transform(bow_transformer.transform([train['text'][1230]])))

  (0, 3019)	0.4700546547381698
  (0, 2699)	0.34618951012493654
  (0, 2249)	0.22878569166423807
  (0, 2100)	0.43926482529741256
  (0, 1049)	0.4700546547381698
  (0, 327)	0.43926482529741256


In [300]:
tfidf_transformer.idf_[bow_transformer.vocabulary_['cladding']]

8.543405753193557

In [301]:
tfidf_text = tfidf_transformer.transform(bow_text)

In [302]:
tfidf_text.shape

(7552, 6693)

**ML Algorithm Selection**

In [303]:
train['target'].value_counts()

0    4323
1    3229
Name: target, dtype: int64

In [304]:
# So, a classification algorithm eg. Logistic regression, naive bayes

In [305]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()

In [306]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tfidf_text, train['target'], test_size=0.3, random_state=101)

In [307]:
from sklearn.metrics import confusion_matrix, classification_report

In [308]:
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)

nb.fit(X_train, y_train)
nb_pred = nb.predict(X_test)

rfc.fit(X_train, y_train)
rfc_pred = rfc.predict(X_test)

/home/lyrax/.local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/lyrax/.local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [309]:
print("---------------------------------")
print("Logistic Regression")
print("---------------------------------")
print(confusion_matrix(y_test, lr_pred))
print(classification_report(y_test, lr_pred))
print("---------------------------------")
print("Naive Bayes MultinomialNB")
print("---------------------------------")
print(confusion_matrix(y_test, nb_pred))
print(classification_report(y_test, nb_pred))
print("---------------------------------")
print("RandomForestClassifier")
print("---------------------------------")
print(confusion_matrix(y_test, rfc_pred))
print(classification_report(y_test, rfc_pred))

---------------------------------
Logistic Regression
---------------------------------
[[1162  165]
 [ 299  640]]
              precision    recall  f1-score   support

           0       0.80      0.88      0.83      1327
           1       0.80      0.68      0.73       939

    accuracy                           0.80      2266
   macro avg       0.80      0.78      0.78      2266
weighted avg       0.80      0.80      0.79      2266

---------------------------------
Naive Bayes MultinomialNB
---------------------------------
[[1136  191]
 [ 299  640]]
              precision    recall  f1-score   support

           0       0.79      0.86      0.82      1327
           1       0.77      0.68      0.72       939

    accuracy                           0.78      2266
   macro avg       0.78      0.77      0.77      2266
weighted avg       0.78      0.78      0.78      2266

---------------------------------
RandomForestClassifier
---------------------------------
[[1106  221]
 [ 325

In [310]:
# Logistic regression did quite well than the other two

In [311]:
# break()
qedxqwgefqdsfewg

NameError: name 'qedxqwgefqdsfewg' is not defined

**Building a data pipeline to predict on the test data.**

In [ ]:
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ("bow cv", CountVectorizer(analyzer=text_process)),
    ("tfidf", TfidfTransformer()),
    ("lr classifier", LogisticRegression())
])

In [ ]:
text_train, text_test, label_train, label_test = train_test_split(train['text'], train['target'],
                                                                  test_size=0.3, random_state=101) 

In [ ]:
pipe.fit(text_train, label_train)

In [ ]:
pipe_pred = pipe.predict(test['text'])

In [ ]:
pipe_pred.shape

In [ ]:
test.shape

In [ ]:
submission_df = pd.DataFrame({
    "id": test['id'],
    "target": pipe_pred
})

In [ ]:
submission_df

In [ ]:
#submission_df.to_csv('submission_df.csv', index=False)